In [1]:
import datasets as ds
from MNIST_functions import *

In [2]:
permuted_train_loaders, permuted_test_loaders, rotated_train_loaders, rotated_test_loaders = ds.load_datasets()


In [3]:
run_experiment_2A_2C('2A', permuted_train_loaders, permuted_test_loaders)

NameError: name 'run_experiment_2A_2C' is not defined